In [56]:
%pip install transformers
%pip install nltk
%pip install scipy
%pip install scikit-learn scipy matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)

[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages

In [32]:
import re
from typing import Set
from transformers import GPT2TokenizerFast

import numpy as np
from nltk.tokenize import sent_tokenize
from pathlib import Path
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd

In [33]:
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

def count_tokens(text: str) -> int:
  """count the number of tokens in a string"""
  return len(tokenizer.encode(text))

def reduce_long(
  long_text: str, long_text_tokens: bool = False, max_len: int = 590
) -> str:
  """
  Reduce a long text to a maximum of `max_len` tokens by potentially cutting at a sentence end
  """
  if not long_text_tokens:
    long_text_tokens = count_tokens(long_text)
  if long_text_tokens > max_len:
    sentences = sent_tokenize(long_text.replace("\n", " "))
    ntokens = 0
    for i, sentence in enumerate(sentences):
      ntokens += 1 + count_tokens(sentence)
      if ntokens > max_len:
        return ". ".join(sentences[:i]) + "."

  return long_text

In [34]:
trainingData = list(Path("training/facts/").glob("**/*.*"))

textSplitter = CharacterTextSplitter(chunk_size=1000, separator="\n\n", chunk_overlap  = 200)

data = []
for training in trainingData:
  with open(training) as f:
    # print(f"Add {f.name} to dataset")
    data.append(f.read())
docs = []
for sets in data:
  docs.extend(textSplitter.split_text(sets))

print(docs)

Created a chunk of size 1439, which is longer than the specified 1000
Created a chunk of size 1193, which is longer than the specified 1000
Created a chunk of size 1982, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1641, which is longer than the specified 1000
Created a chunk of size 1144, which is longer than the specified 1000
Created a chunk of size 1549, which is longer than the specified 1000
Created a chunk of size 1305, which is longer than the specified 1000
Created a chunk of size 1447, which is longer than the specified 1000
Created a chunk of size 1660, which is longer than the specified 1000
Created a chunk of size 1488, which is longer than the specified 1000
Created a chunk of size 3401, which is longer than the specified 1000
Created a chunk of size 1106, which is longer than the specified 1000
Created a chunk of size 3183, which is longer than the specified 1000
Created a chunk of s

['Heart to heart with rookie board members', 'I got appointed as a director on a company’s board of directors for the first time in January 2012, I was 29 years old and had heard the term fiduciary duty for the very first time.\nThe company in question was Inbox Business Technologies, Pakistan’s largest system integrator / technology services company by revenue. My appointment happened because Dawood Hercules Group (DH) owned a majority stake in Inbox and I was working with DH to launch a private equity firm called Cyan.\nI remember being overwhelmed, not knowing where to turn for advice. The existing set of directors who I had interacted with didn’t have a uniform understanding of what it meant to be a good director. So I naturally turned to the web as I do for everything else. I came across two books I) Boards that Lead and II) CEO’s Boss. While the text was prescriptive and relevant I had a really hard time figuring out how to draw the line between “being strategic” and “being too o

In [35]:
res = []
for content in docs:
    res += [(content, count_tokens(content))]
df = pd.DataFrame(res, columns=["content", "tokens"])
df = df[df.tokens>40]
df = df.reset_index().drop('index',axis=1)
df.head()

,content,tokens
0,I got appointed as a director on a company’s b...,318
1,1. Read the board materials thoroughly and mak...,103
2,2. Invest time in really getting to know the C...,103
3,3. Get to know the company’s customers and com...,119
4,4. “Grilling” Management or “making a point” i...,144


In [36]:
df.to_csv('training/content.csv', index=False)

create questions

In [37]:
import ast  # for converting embeddings saved as strings back to arrays
import openai  # for calling the OpenAI API
import pandas as pd  # for storing text and embeddings data
import tiktoken  # for counting tokens
from scipy import spatial  # for calculating vector similarities for search
import os


# models
EMBEDDING_MODEL = "text-embedding-ada-002"
GPT_MODEL = "gpt-3.5-turbo"

from dotenv import load_dotenv

load_dotenv()
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
# query = 'Which athletes won the gold medal in curling at the 2022 Winter Olympics?'

# response = openai.ChatCompletion.create(
#     messages=[
#         {'role': 'system', 'content': 'You answer questions about the 2022 Winter Olympics.'},
#         {'role': 'user', 'content': query},
#     ],
#     model=GPT_MODEL,
#     temperature=0,
# )

# print(response['choices'][0]['message']['content'])

In [40]:
import openai

def get_questions(context):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"Form questions to me based on the text below \n\nText: {context}\n\nQuestions:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        print(context, response['choices'][0]['text'])
        return response['choices'][0]['text']
    except:
        return ""

# df['context'] = df.title + "\n" + df.heading + "\n\n" + df.content
df['context'] = df.content
df['questions']= df.context.apply(get_questions)
df['questions'] = "1." + df.questions
print(df[['questions']].values[0][0])

I got appointed as a director on a company’s board of directors for the first time in January 2012, I was 29 years old and had heard the term fiduciary duty for the very first time.
The company in question was Inbox Business Technologies, Pakistan’s largest system integrator / technology services company by revenue. My appointment happened because Dawood Hercules Group (DH) owned a majority stake in Inbox and I was working with DH to launch a private equity firm called Cyan.
I remember being overwhelmed, not knowing where to turn for advice. The existing set of directors who I had interacted with didn’t have a uniform understanding of what it meant to be a good director. So I naturally turned to the web as I do for everything else. I came across two books I) Boards that Lead and II) CEO’s Boss. While the text was prescriptive and relevant I had a really hard time figuring out how to draw the line between “being strategic” and “being too operational”.
In the years to come DH ended up ap

In [20]:
df.to_csv('training/questions.csv', index=False)

In [41]:
def get_answers(row):
    try:
        response = openai.Completion.create(
            engine="davinci-instruct-beta-v3",
            prompt=f"Write answer based on the text below\n\nText: {row.context}\n\nQuestions:\n{row.questions}\n\nAnswers:\n1.",
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response['choices'][0]['text']
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1." + df.answers
df = df.dropna().reset_index().drop('index',axis=1)
print(df[['answers']].values[0][0])

1. A fiduciary duty is a responsibility that a director has to the company that they are a director of.
2. The author turned to the web for advice when he first became a director.
3. The author learned that it is important to be strategic but not too operational.
4. The author would advise first time directors to be strategic but not too operational.


In [42]:
df.to_csv('training/qa.csv', index=False)

In [43]:
df = pd.read_csv('training/qa.csv')

In [44]:
df.head()

,content,tokens,context,questions,answers
0,I got appointed as a director on a company’s b...,318,I got appointed as a director on a company’s b...,1. What is a fiduciary duty?\n2. What did the ...,1. A fiduciary duty is a responsibility that a...
1,1. Read the board materials thoroughly and mak...,103,1. Read the board materials thoroughly and mak...,1.,1. Boards need to do their part and it starts ...
2,2. Invest time in really getting to know the C...,103,2. Invest time in really getting to know the C...,1. What is the group mentality like during boa...,1. The group mentality during board meetings i...
3,3. Get to know the company’s customers and com...,119,3. Get to know the company’s customers and com...,1. What is the main purpose of board meetings?...,1. The main purpose of board meetings is for M...
4,4. “Grilling” Management or “making a point” i...,144,4. “Grilling” Management or “making a point” i...,1.,1. The author describes how they changed their...


In [59]:
df = pd.read_csv('training/qa.csv')
olympics_search_fileid = "file-c3shd8wqF3vSCKaukW4Jr1TT"
df.head()

,content,tokens,context,questions,answers
0,I got appointed as a director on a company’s b...,318,I got appointed as a director on a company’s b...,1. What is a fiduciary duty?\n2. What did the ...,1. A fiduciary duty is a responsibility that a...
1,1. Read the board materials thoroughly and mak...,103,1. Read the board materials thoroughly and mak...,1.,1. Boards need to do their part and it starts ...
2,2. Invest time in really getting to know the C...,103,2. Invest time in really getting to know the C...,1. What is the group mentality like during boa...,1. The group mentality during board meetings i...
3,3. Get to know the company’s customers and com...,119,3. Get to know the company’s customers and com...,1. What is the main purpose of board meetings?...,1. The main purpose of board meetings is for M...
4,4. “Grilling” Management or “making a point” i...,144,4. “Grilling” Management or “making a point” i...,1.,1. The author describes how they changed their...


In [60]:
import random

def get_random_similar_contexts(question, context, file_id=olympics_search_fileid, search_model='ada', max_rerank=10):
    """
    Find similar contexts to the given context using the search file
    """
    try:
        results = openai.Engine(search_model).search(
            search_model=search_model, 
            query=question, 
            max_rerank=max_rerank,
            file=file_id
        )
        candidates = []
        for result in results['data'][:3]:
            if result['text'] == context:
                continue
            candidates.append(result['text'])
        random_candidate = random.choice(candidates)
        return random_candidate
    except Exception as e:
        print(e)
        return ""
    
def create_fine_tuning_dataset(df, discriminator=False, n_negative=1, add_related=False):
    """
    Create a dataset for fine tuning the OpenAI model; either for a discriminator model, 
    or a model specializing in Q&A, where it says if no relevant context is found.

    Parameters
    ----------
    df: pd.DataFrame
        The dataframe containing the question, answer and context pairs
    discriminator: bool
        Whether to create a dataset for the discriminator
    n_negative: int
        The number of random negative samples to add (using a random context)
    add_related: bool
        Whether to add the related contexts to the correct context. These are hard negative examples

    Returns
    -------
    pd.DataFrame
        The dataframe containing the prompts and completions, ready for fine-tuning
    """
    rows = []
    for i, row in df.iterrows():
        for q, a in zip(("1." + row.questions).split('\n'), ("1." + row.answers).split('\n')):
            if len(q) >10 and len(a) >10:
                if discriminator:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" yes"})
                else:
                    rows.append({"prompt":f"{row.context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" {a[2:].strip()}"})

    for i, row in df.iterrows():
        for q in ("1." + row.questions).split('\n'):
            if len(q) >10:
                for j in range(n_negative + (2 if add_related else 0)):
                    random_context = ""
                    if j == 0 and add_related:
                        # add the related contexts based on originating from the same wikipedia page
                        # subset = df[(df.title == row.title) & (df.context != row.context)]
                        subset = df[(df.context != row.context)]
                        
                        if len(subset) < 1:
                            continue
                        random_context = subset.sample(1).iloc[0].context
                    if j == 1 and add_related:
                        # add the related contexts based on the most similar contexts according to the search
                        random_context = get_random_similar_contexts(q[2:].strip(), row.context, search_model='ada', max_rerank=10)
                    else:
                        while True:
                            # add random context, which isn't the correct context
                            random_context = df.sample(1).iloc[0].context
                            if random_context != row.context:
                                break
                    if discriminator:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\n Related:", "completion":f" no"})
                    else:
                        rows.append({"prompt":f"{random_context}\nQuestion: {q[2:].strip()}\nAnswer:", "completion":f" No appropriate context found to answer the question."})

    return pd.DataFrame(rows) 


In [61]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
len(train_df), len(test_df)

(32, 9)

In [62]:
for name, is_disc in [('discriminator', True), ('qa', False)]:
    for train_test, dt in [('train', train_df), ('test', test_df)]:
        ft = create_fine_tuning_dataset(dt, discriminator=is_disc, n_negative=1, add_related=True)
        ft.to_json(f'{name}_{train_test}.jsonl', orient='records', lines=True)

AttributeError: 'DataFrame' object has no attribute 'title'